# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 28 2022 3:30PM,249633,16,8704489,Sartorius Lab Products and Service,Released
1,Oct 28 2022 3:20PM,249632,16,8703933,Sartorius Bioprocess Solutions,Released
2,Oct 28 2022 2:49PM,249629,10,CTF0011472,"Citieffe, Inc.",Executing
3,Oct 28 2022 1:52PM,249621,10,PRF-40532,Bio-PRF,Executing
4,Oct 28 2022 1:52PM,249621,10,PRF-40535,Bio-PRF,Completed
5,Oct 28 2022 1:52PM,249621,10,PRF-40538,Bio-PRF,Executing
6,Oct 28 2022 1:52PM,249621,10,PRF-40541,Bio-PRF,Executing
7,Oct 28 2022 10:16AM,249594,12,8703768,"SD Head USA, LLC",Released
8,Oct 28 2022 10:14AM,249593,15,ALI029348261,"Alliance Pharma, Inc.",Completed
9,Oct 28 2022 10:14AM,249593,15,30011791,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
9,249621,Completed,1
10,249621,Executing,3
11,249629,Executing,1
12,249632,Released,1
13,249633,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249594,NaN,NaN,1.0
249621,1.0,3.0,NaN
249629,NaN,1.0,NaN
249632,NaN,NaN,1.0
249633,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249572,21.0,0.0,7.0
249587,4.0,0.0,1.0
249592,0.0,0.0,1.0
249593,1.0,1.0,1.0
249594,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249572,21,0,7
249587,4,0,1
249592,0,0,1
249593,1,1,1
249594,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249572,21,0,7
1,249587,4,0,1
2,249592,0,0,1
3,249593,1,1,1
4,249594,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249572,21,,7
1,249587,4,,1
2,249592,,,1
3,249593,1,1,1
4,249594,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 28 2022 3:30PM,249633,16,Sartorius Lab Products and Service
1,Oct 28 2022 3:20PM,249632,16,Sartorius Bioprocess Solutions
2,Oct 28 2022 2:49PM,249629,10,"Citieffe, Inc."
3,Oct 28 2022 1:52PM,249621,10,Bio-PRF
7,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC"
8,Oct 28 2022 10:14AM,249593,15,"Alliance Pharma, Inc."
11,Oct 28 2022 10:14AM,249592,15,"Brookfield Pharmaceuticals, LLC"
12,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service
17,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 28 2022 3:30PM,249633,16,Sartorius Lab Products and Service,,,1
1,Oct 28 2022 3:20PM,249632,16,Sartorius Bioprocess Solutions,,,1
2,Oct 28 2022 2:49PM,249629,10,"Citieffe, Inc.",,1,
3,Oct 28 2022 1:52PM,249621,10,Bio-PRF,1,3,
4,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",,,1
5,Oct 28 2022 10:14AM,249593,15,"Alliance Pharma, Inc.",1,1,1
6,Oct 28 2022 10:14AM,249592,15,"Brookfield Pharmaceuticals, LLC",,,1
7,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,4,,1
8,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",21,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 3:30PM,249633,16,Sartorius Lab Products and Service,1,,
1,Oct 28 2022 3:20PM,249632,16,Sartorius Bioprocess Solutions,1,,
2,Oct 28 2022 2:49PM,249629,10,"Citieffe, Inc.",,1,
3,Oct 28 2022 1:52PM,249621,10,Bio-PRF,,3,1
4,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",1,,
5,Oct 28 2022 10:14AM,249593,15,"Alliance Pharma, Inc.",1,1,1
6,Oct 28 2022 10:14AM,249592,15,"Brookfield Pharmaceuticals, LLC",1,,
7,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,1,,4
8,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",7,,21


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 3:30PM,249633,16,Sartorius Lab Products and Service,1,,
1,Oct 28 2022 3:20PM,249632,16,Sartorius Bioprocess Solutions,1,,
2,Oct 28 2022 2:49PM,249629,10,"Citieffe, Inc.",,1,
3,Oct 28 2022 1:52PM,249621,10,Bio-PRF,,3,1
4,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 3:30PM,249633,16,Sartorius Lab Products and Service,1.0,NaN,NaN
1,Oct 28 2022 3:20PM,249632,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Oct 28 2022 2:49PM,249629,10,"Citieffe, Inc.",NaN,1.0,NaN
3,Oct 28 2022 1:52PM,249621,10,Bio-PRF,NaN,3.0,1.0
4,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 3:30PM,249633,16,Sartorius Lab Products and Service,1.0,0.0,0.0
1,Oct 28 2022 3:20PM,249632,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Oct 28 2022 2:49PM,249629,10,"Citieffe, Inc.",0.0,1.0,0.0
3,Oct 28 2022 1:52PM,249621,10,Bio-PRF,0.0,3.0,1.0
4,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,499250,0.0,4.0,1.0
12,249594,1.0,0.0,0.0
15,748757,9.0,1.0,22.0
16,748852,3.0,0.0,4.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,499250,0.0,4.0,1.0
1,12,249594,1.0,0.0,0.0
2,15,748757,9.0,1.0,22.0
3,16,748852,3.0,0.0,4.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,0.0,4.0,1.0
1,12,1.0,0.0,0.0
2,15,9.0,1.0,22.0
3,16,3.0,0.0,4.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,0.0
1,12,Released,1.0
2,15,Released,9.0
3,16,Released,3.0
4,10,Executing,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16
Status,,,,
Completed,1.0,0.0,22.0,4.0
Executing,4.0,0.0,1.0,0.0
Released,0.0,1.0,9.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16
0,Completed,1.0,0.0,22.0,4.0
1,Executing,4.0,0.0,1.0,0.0
2,Released,0.0,1.0,9.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16
0,Completed,1.0,0.0,22.0,4.0
1,Executing,4.0,0.0,1.0,0.0
2,Released,0.0,1.0,9.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()